<a href="https://colab.research.google.com/github/stevejj4/Data-Cleaning/blob/main/SUN_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default

# Authorize Google Colab
auth.authenticate_user()

# Credentials for Google Sheets
creds, _ = default()
gc = gspread.authorize(creds)

# Load the Google Sheet
spreadsheet = gc.open('SUN_Dashboard')

# Check if the sheet 'transformed' exists and create it if not
try:
    transformed_sheet = spreadsheet.worksheet('transformed')
except gspread.exceptions.WorksheetNotFound:
    transformed_sheet = spreadsheet.add_worksheet(title='transformed', rows="1000", cols="20")

# Load the original data
worksheet = spreadsheet.sheet1
rows = worksheet.get_all_values()

# Create DataFrame from the sheet data
df = pd.DataFrame(rows)
df.columns = df.iloc[0]  # Set the first row as column names
df = df.iloc[1:]  # Remove the first row


In [19]:
df.shape

(68, 21)

## Explanatory data analysis(EDA)
The graphical exploration/ representation of data (1977, John Turkey). From this we are able to form hypothesess for our data for further analyses or study
‘Exploratory data analysis’ is an attitude, a state of flexibility, a willingness to look for those things that we believe are not there, as well as those we believe to be there.”

In [20]:
# Reshape the data from wide to long format
value_vars = ['Contribution', 'Compensation', 'Deaths']
months = ['January', 'February', 'March', 'April', 'May']

# Initialize an empty list to store the reshaped data
reshaped_data = []

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    for month in months:
        reshaped_data.append({
            'PAYBILL NO': row['PAYBILL NO'],
            'SUB COUNTY': row['SUB COUNTY'],
            'REGION': row['REGION'],
            'Cluster': row['Cluster'],
            'County': row['County'],
            'Site': row['Site'],
            'Month': month,
            'Contribution': row[f'{month}_Contribution'],
            'Compensation': row[f'{month}_Compensation'],
            'Deaths_per_Month': row[f'{month}_deaths']
        })

# Create a new DataFrame from the reshaped data
reshaped_df = pd.DataFrame(reshaped_data)

# Convert the DataFrame to a list of lists
reshaped_data_list = [reshaped_df.columns.values.tolist()] + reshaped_df.values.tolist()

# Clear the 'transformed' sheet
transformed_sheet.clear()

# Write the reshaped DataFrame to the 'transformed' sheet
transformed_sheet.update('A1', reshaped_data_list)

# Print confirmation
print('Data successfully loaded into the transformed sheet.')


<ipython-input-20-4bb9bf779074>:34: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  transformed_sheet.update('A1', reshaped_data_list)


Data successfully loaded into the transformed sheet.


In [35]:
# Select the sheet named 'transformed'
worksheet = spreadsheet.worksheet('transformed')

# Get all the values from the sheet
rows = worksheet.get_all_values()

# Convert the data to a pandas DataFrame
df = pd.DataFrame(rows)

# Set the first row as the header
df.columns = df.iloc[0]
df = df.drop(0)

,PAYBILL NO,SUB COUNTY,REGION,Cluster,County,Site,Month,Contribution,Compensation,Deaths_per_Month
1,4068299,KIBERA,Nairobi,1,Nairobi,Sub County,January,"976,470.00","610,000.00",13.00
2,4068299,KIBERA,Nairobi,1,Nairobi,Sub County,February,"755,400.00","660,000.00",14.00
3,4068299,KIBERA,Nairobi,1,Nairobi,Sub County,March,"742,030.00","740,000.00",16.00
4,4068299,KIBERA,Nairobi,1,Nairobi,Sub County,April,"717,701.00","940,000.00",20.00
5,4068299,KIBERA,Nairobi,1,Nairobi,Sub County,May,"851,222.00","250,000.00",5


In [36]:
#cheking the data type
df.dtypes

0
PAYBILL NO          object
SUB COUNTY          object
REGION              object
Cluster             object
County              object
Site                object
Month               object
Contribution        object
Compensation        object
Deaths_per_Month    object
dtype: object

In [ ]:
# Check if there maybe any missing data
df.isnull().sum()

In [ ]:
# “Graphs force us to note the unexpected; nothing could be more important.”
#Lets start by analyzing the monthly trends for contributions, compensation amounts and the number of deaths
#Converting the Month column to a categorical type with a proper order
# Assuming transformed_data is your DataFrame

import pandas as pd
import matplotlib.pyplot as plt

# Load the transformed data
df = pd.read_csv('transformed_data.csv')

# Convert 'Month' to a datetime object for proper sorting
df['Month'] = pd.to_datetime(df['Month'], format='%b')

# Sort the DataFrame by Month
df = df.sort_values(by='Month')

# Plotting trends
plt.figure(figsize=(14, 8))
for col in ['Contribution', 'Compensation', 'Deaths_per_Month']:
    plt.plot(df['Month'], df[col], label=col)
plt.xlabel('Month')
plt.ylabel('Values')
plt.title('Trend Analysis')
plt.legend()
plt.grid(True)
plt.show()
